In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class MultiClassPerceptron:
    def __init__(self, learning_rate=0.1, n_iterations=100):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))

        # Initialize weights and bias for each class
        self.weights = np.zeros((n_classes, n_features))
        self.bias = np.zeros(n_classes)

        # Convert labels to binary for each class (one-vs-rest)
        y_binary = np.array([self._to_binary(y, c) for c in range(n_classes)])

        # Training loop
        for epoch in range(self.n_iterations):
            for class_idx in range(n_classes):
                linear_output = np.dot(X, self.weights[class_idx]) + self.bias[class_idx]
                y_predicted = self.tlu(linear_output)

                # Update weights and bias using vectorized operations
                errors = y_binary[class_idx] - y_predicted
                self.weights[class_idx] += self.learning_rate * np.dot(X.T, errors)
                self.bias[class_idx] += self.learning_rate * np.sum(errors)

    def predict(self, X):
        # Compute linear output for each class
        linear_output = np.array([np.dot(X, self.weights[c]) + self.bias[c] for c in range(self.weights.shape[0])])

        # Apply threshold linear unit (TLU) function to get predictions for each class
        y_predicted = np.array([self.tlu(linear_output[c]) for c in range(self.weights.shape[0])])

        # Return the class with the highest prediction score
        return np.argmax(y_predicted, axis=0)

    def tlu(self, x):
        # Threshold Linear Unit (TLU) function
        return np.where(x >= 0, 1, 0)

    def _to_binary(self, y, class_idx):
        # Convert labels to binary for one-vs-rest classification
        return np.where(y == class_idx, 1, 0)

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the Multi-Class Perceptron
perceptron = MultiClassPerceptron(learning_rate=0.1, n_iterations=100)
perceptron.fit(X_train, y_train)

# Test the trained Perceptron
print("Testing the trained Perceptron:")
predictions = perceptron.predict(X_test)
for inputs, true_label, prediction in zip(X_test, y_test, predictions):
    print(f"Input: {inputs}, True Label: {true_label}, Predicted: {prediction} -> {'Correct' if true_label == prediction else 'Incorrect'}")

# Calculate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


Testing the trained Perceptron:
Input: [ 0.35451684 -0.58505976  0.55777524  0.02224751], True Label: 1, Predicted: 1 -> Correct
Input: [-0.13307079  1.65083742 -1.16139502 -1.17911778], True Label: 0, Predicted: 0 -> Correct
Input: [ 2.30486738 -1.0322392   1.8185001   1.49058286], True Label: 2, Predicted: 1 -> Incorrect
Input: [ 0.23261993 -0.36147005  0.44316389  0.4227026 ], True Label: 1, Predicted: 0 -> Incorrect
Input: [ 1.2077952  -0.58505976  0.61508092  0.28921757], True Label: 1, Predicted: 1 -> Correct
Input: [-0.49876152  0.75647855 -1.27600637 -1.04563275], True Label: 0, Predicted: 0 -> Correct
Input: [-0.2549677  -0.36147005 -0.07258719  0.15573254], True Label: 1, Predicted: 0 -> Incorrect
Input: [1.32969211 0.08570939 0.78699794 1.49058286], True Label: 2, Predicted: 2 -> Correct
Input: [ 0.47641375 -1.92659808  0.44316389  0.4227026 ], True Label: 1, Predicted: 1 -> Correct
Input: [-0.01117388 -0.80864948  0.09932984  0.02224751], True Label: 1, Predicted: 1 -> Corr